In [1]:
import tensorflow as tf

/Users/xuelun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras

Using TensorFlow backend.


In [3]:
print(keras.__version__)


2.1.6


In [7]:
print(tf.__version__)

1.9.0


In [7]:
from keras.models import Sequential

model = Sequential()


In [8]:
from keras.layers.embeddings import Embedding
model.add(Embedding(50922+1, 256))


In [9]:
from keras.layers.recurrent import LSTM, GRU
model.add(LSTM(128)) # try using a GRU instead, for fun



In [10]:
from keras.layers.core import Dense, Dropout, Activation

model.add(Dropout(0.5))

In [11]:
model.add(Dense(1))

In [12]:
model.add(Activation('sigmoid'))

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [22]:
import pandas as pd
import jieba
#neg=pd.read_excel('neg.xls',header=None,index=None)
#pos=pd.read_excel('pos.xls',header=None,index=None) #读取训练语料完毕
content_tag=pd.read_excel('comment_tag1.xls',header=None,index=None)


1       0
2       1
3       1
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
30      0
       ..
2373    0
2374    0
2375    0
2376    0
2377    0
2378    0
2379    0
2380    1
2381    1
2382    0
2383    0
2384    0
2385    0
2386    0
2387    1
2388    0
2389    1
2390    1
2391    1
2392    1
2393    1
2394    0
2395    1
2396    0
2397    1
2398    1
2399    1
2400    0
2401    0
2402    1
Name: 1, Length: 2402, dtype: object

In [60]:

content_tag['mark']=content_tag[1:].iloc[:,1]
cw = lambda x: list(jieba.cut(x))
content_tag['words'] = content_tag[0].apply(cw)
comment = pd.read_excel('comment.xls') #读入评论内容
comment['words'] = comment['comment'].apply(cw) #评论分词 
d2v_train = pd.concat([content_tag['words'], comment['words']], ignore_index = True) 

w = [] #将所有词语整合在一起
for i in d2v_train:
    w.extend(i)
dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数

del w,d2v_train
dict['id']=list(range(1,len(dict)+1))

get_sent = lambda x: list(dict['id'][x])
content_tag['sent'] = content_tag['words'].apply(get_sent) #速度太慢，将分的词换成计算频次dataframe中的行序号，默认是按频次从高到低排序的，即序号是1出现次数最多
comment['sent'] = comment['words'].apply(get_sent) #速度太慢，将分的词换成计算频次dataframe中的行序号，默认是按频次从高到低排序的，即序号是1出现次数最多
from keras.preprocessing import sequence

maxlen = 6#输入词序列长度不一样，只截取每个句子切成词序列后的2个词标，计算

print("Pad sequences (samples x time)")
content_tag['sent'] = list(sequence.pad_sequences(content_tag['sent'], maxlen=maxlen))#每一行的词个数变成50，pad_sequences可以自己检测选择重要（出现次数多）的词？
comment['sent'] = list(sequence.pad_sequences(comment['sent'], maxlen=maxlen))#每一行的词个数变成50，pad_sequences可以自己检测选择重要（出现次数多）的词？

import numpy as np 
train_x=np.array(list(content_tag['sent'][1:]))
train_y=np.array(list(content_tag[1:].iloc[:,1]))

#train_y=np.array(list(content_tag['mark']))
test_x=np.array(list(comment['sent']))


Pad sequences (samples x time)


array([[   0,    0,    0,  131,   53,    5],
       [   0,    0,    0,    0,    0,   35],
       [   1,    1,   34,  233, 1323,    6],
       ...,
       [   0,    0,    0,   31,   51,   32],
       [   0,    0,    0,    0,    0,  326],
       [ 280,   10,  369,  259,    3,   15]], dtype=int32)

In [46]:
content_tag['sent'] = content_tag['words'].apply(get_sent) #速度太慢，将分的词换成计算频次dataframe中的行序号，默认是按频次从高到低排序的，即序号是1出现次数最多


In [61]:
content_tag['sent'].iloc[1021]

array([  3, 111,  87,   1, 111, 726], dtype=int32)

In [48]:
content_tag['sent'] = list(sequence.pad_sequences(content_tag['sent'], maxlen=8))#每一行的词个数变成50，pad_sequences可以自己检测选择重要（出现次数多）的词？


In [62]:
content_tag['sent'].iloc[1021]

array([  3, 111,  87,   1, 111, 726], dtype=int32)

In [100]:
model.fit(train_x, train_y, batch_size=16, nb_epoch=10) #训练时间为若干个小时

/Users/xuelun/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
2402/2402 [==============================] - 45s 19ms/step - loss: nan - acc: 0.0000e+00
Epoch 2/10
2402/2402 [==============================] - 43s 18ms/step - loss: nan - acc: 0.0000e+00
Epoch 3/10
2402/2402 [==============================] - 38s 16ms/step - loss: nan - acc: 0.0000e+00
Epoch 4/10
2402/2402 [==============================] - 36s 15ms/step - loss: nan - acc: 0.0000e+00
Epoch 5/10
2402/2402 [==============================] - 42s 17ms/step - loss: nan - acc: 0.0000e+00
Epoch 6/10
2402/2402 [==============================] - 46s 19ms/step - loss: nan - acc: 0.0000e+00
Epoch 7/10
2402/2402 [==============================] - 52s 21ms/step - loss: nan - acc: 0.0000e+00
Epoch 8/10
2402/2402 [==============================] - 52s 22ms/step - loss: nan - acc: 0.0000e+00
Epoch 9/10
2402/2402 [==============================] - 49s 20ms/step - loss: nan - acc: 0.0000e+00
Epoch 10/10
2402/2402 [==============================] - 49s 21ms/step - loss: nan - acc: 0.0000e+00

In [68]:
classes = model.predict_classes(train_x)
#acc = np_utils.accuracy(classes, yt)
#print('Test accuracy:', acc)

/Users/xuelun/anaconda3/lib/python3.6/site-packages/keras/models.py:1181: RuntimeWarning: invalid value encountered in greater
  return (proba > 0.5).astype('int32')


In [81]:
from keras.utils import np_utils 
count=1
for i in len(train_y):
    if classes[i][0]==train_y[i][0]:
        count=count+1 
print(count/len(train_y))

        


TypeError: 'int' object is not iterable

In [93]:
train_y=np.array(list(content_tag[1:].iloc[:,1]))



In [94]:
train_y


array([0, 1, 1, ..., 0, 0, 1])

In [103]:
loss, accuracy = model.evaluate(train_x, train_y)

#acc = np_utils.accuracy(classes, train_y)
#print('Test accuracy:', acc)

2402/2402 [==============================] - 1s 376us/step


In [105]:
print(loss)

nan


In [23]:

#neg['mark']=0 #给训练语料贴上标签
#pn=pd.concat([pos,neg],ignore_index=True) #合并语料
#neglen=len(neg)
#poslen=len(pos) #计算语料数目
#cw = lambda x: list(jieba.cut(x)) #定义分词函数
#pn['words'] = pn[0].apply(cw)#pn[0]是原始从excel读出来的
#cw = lambda x: list(jieba.cut(x)) #定义分词函数
#pn['words'] = pn[0].apply(cw)#pn[0]是原始从excel读出来的

#comment = pd.read_excel('sum.xls') #读入评论内容
comment = pd.read_excel('comment.xls') #读入评论内容

#comment = pd.read_csv('a.csv', encoding='utf-8')
#comment = comment[comment['rateContent'].notnull()] #仅读取非空评论
comment['words'] = comment['comment'].apply(cw) #评论分词 

d2v_train = pd.concat([pn['words'], comment['words']], ignore_index = True) 

w = [] #将所有词语整合在一起
for i in d2v_train:
    w.extend(i)
dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/m2/vy0jdnls50x9z9zr96vrqmj40000gn/T/jieba.cache
Loading model cost 0.925 seconds.
Prefix dict has been built succesfully.


In [25]:
del w,d2v_train
dict['id']=list(range(1,len(dict)+1))

get_sent = lambda x: list(dict['id'][x])
pn['sent'] = pn['words'].apply(get_sent) #速度太慢，将分的词换成计算频次dataframe中的行序号，默认是按频次从高到低排序的，即序号是1出现次数最多
comment['sent'] = comment['words'].apply(get_sent) #速度太慢，将分的词换成计算频次dataframe中的行序号，默认是按频次从高到低排序的，即序号是1出现次数最多

In [27]:
from keras.preprocessing import sequence

maxlen = 2#输入词序列长度不一样，只截取每个句子切成词序列后的2个词标，计算

print("Pad sequences (samples x time)")
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen))#每一行的词个数变成50，pad_sequences可以自己检测选择重要（出现次数多）的词？
comment['sent'] = list(sequence.pad_sequences(comment['sent'], maxlen=maxlen))#每一行的词个数变成50，pad_sequences可以自己检测选择重要（出现次数多）的词？

Pad sequences (samples x time)


In [29]:
import numpy as np 
train_x=np.array(list(pn['sent']))
train_y=np.array(list(pn['mark']))
test_x=np.array(list(comment['sent']))

In [35]:
len(train_x)

21105

In [106]:
neg=pd.read_excel('neg.xls',header=None,index=None)
pos=pd.read_excel('pos.xls',header=None,index=None) #读取训练语料完毕
pos['mark']=1
neg['mark']=0 #给训练语料贴上标签
pn=pd.concat([pos,neg],ignore_index=True) #合并语料
neglen=len(neg)
poslen=len(pos) #计算语料数目

cw = lambda x: list(jieba.cut(x)) #定义分词函数
pn['words'] = pn[0].apply(cw)

comment = pd.read_excel('sum.xls') #读入评论内容
#comment = pd.read_csv('a.csv', encoding='utf-8')
comment = comment[comment['rateContent'].notnull()] #仅读取非空评论
comment['words'] = comment['rateContent'].apply(cw) #评论分词 

d2v_train = pd.concat([pn['words'], comment['words']], ignore_index = True) 

w = [] #将所有词语整合在一起
for i in d2v_train:
    w.extend(i)

dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数
del w,d2v_train
dict['id']=list(range(1,len(dict)+1))

get_sent = lambda x: list(dict['id'][x])
pn['sent'] = pn['words'].apply(get_sent) #速度太慢

maxlen = 50

print("Pad sequences (samples x time)")
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen))

x = np.array(list(pn['sent']))[::2] #训练集
y = np.array(list(pn['mark']))[::2]
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

print('Build model...')
model = Sequential()
model.add(Embedding(len(dict)+1, 256))
model.add(LSTM(128)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Pad sequences (samples x time)
Build model...


In [108]:
x

array([[   23,  5294,   873, ...,     5, 11034,     3],
       [  906,   182,    13, ...,   599,  2886,     3],
       [  836,     1,    46, ...,    12,   537,     3],
       ...,
       [    0,     0,     0, ...,     9,     9,     9],
       [   23,   225,  1885, ..., 51951, 48102,     9],
       [    0,     0,     0, ...,    15,    19,     3]], dtype=int32)

In [ ]:
model.fit(x, y, batch_size=16, nb_epoch=10) #训练时间为若干个小时


/Users/xuelun/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
10553/10553 [==============================] - 258s 24ms/step - loss: 0.0085 - acc: 0.9976
Epoch 2/10
10553/10553 [==============================] - 220s 21ms/step - loss: 0.0095 - acc: 0.9972
Epoch 3/10
10553/10553 [==============================] - 229s 22ms/step - loss: 0.0058 - acc: 0.9984
Epoch 4/10
 8032/10553 [=====================>........] - ETA: 51s - loss: 0.0012 - acc: 0.9998

In [110]:
classes = model.predict_classes(train_x)
#acc = np_utils.accuracy(classes, yt)
#print('Test accuracy:', acc)

In [122]:
from keras.utils import np_utils 
count=1

for i in range(len(train_y)):
    if classes[i][0]==train_y[i]:
        count=count+1 
print(count/len(train_y))


0.44712739383846795


In [121]:
classes = model.predict_classes(X_test, batch_size=32)

1